In [2]:
import torch
import torch.nn as nn


# 3 차원짜리 임베딩 10개 생성
embedding = nn.Embedding(10, 3)

# [1, 2, 4, 5], [4, 3, 2, 9]에 대한 임베딩 값 구하기
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
print(input.shape)
embedding(input).shape

torch.Size([2, 4])


torch.Size([2, 4, 3])